# Basic analysis

In this notebook, we briefly demonstrate how you can create familair product analyses using Objectiv.

## Getting started

In [ ]:
# import Bach
from bach_open_taxonomy import ObjectivFrame 

In [ ]:
# connect to SQL db
of = ObjectivFrame.from_objectiv_data(db_url='postgresql://@localhost:5433/objectiv',
                                      table_name='data',
                                      time_aggregation='YYYY-MM-DD',
                                      start_date='2022-02-02')

In [ ]:
# adding specific contexts to the data
of['application'] = of.global_contexts.gc.get_from_context_with_type_series(type='ApplicationContext', key='id')
of['root_location'] = of.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
of['referrer'] = of.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='referrer')
of['feature_nice_name'] = of.location_stack.ls.nice_name

In [ ]:
# explore the data
of.sort_values('session_id', ascending=False).head()

In [ ]:
# explore the data with describe
of.describe(include='all').head()

## How many users do we have?

In [ ]:
# model hub: unique users, total month
users = of.model_hub.aggregate.unique_users(time_aggregation='YYYY-MM')
users.sort_index(ascending=False).head()

In [ ]:
# model hub: unique users, daily
users = of.model_hub.aggregate.unique_users()
users.sort_index(ascending=False).head(15)

In [ ]:
# users by RootLocation
users_root = of.groupby(['application', 'root_location']).agg({'user_id':'nunique'})
users_root.sort_values('user_id_nunique', ascending=False).head(10)

## What is their time spent?

In [ ]:
# model hub: duration, total month
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM')
duration.sort_index(ascending=False).head()

In [ ]:
# model hub: duration, daily
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM-DD')
duration.sort_index(ascending=False).head()

In [ ]:
# duration for RootLocation home 
of[(of.root_location == 'home')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

In [ ]:
# duration for RootLocation blog
of[(of.root_location == 'blog')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

In [ ]:
# duration for the whole docs
of[(of.application == 'objectiv-docs')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

In [ ]:
# how is this time spent distributed?

# first calculate the duration of each session
session_duration = of.groupby(['session_id']).agg({'moment':['min', 'max']})
session_duration['duration'] = session_duration['moment_max'] - session_duration['moment_min']

# explore the durations with describe
session_duration['duration'].describe().head()

# materizalize the df
session_duration.materialize(inplace=True)

# define the quantiles 
quantiles = [0.25, 0.50, 0.75]

# distribution of time spemt
session_duration['duration'].quantile(q=quantiles).head()

## What are the top user interactions?

In [ ]:
# select PressEvent and MediaStart event, to focus on user actions
users_feature = of[(of.event_type == 'PressEvent') |
                   (of.event_type == 'MediaStartEvent')]

# users by feature
users_feature = users_feature.groupby(['application', 'feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
users_feature.sort_values('user_id_nunique', ascending=False).head()

## What users do most on home?

In [ ]:
# select homepage RootLocation only
home_users = of[(of.root_location == 'home')]

# select website only and exclude the docs
home_users = home_users[(home_users.application == 'objectiv-website')]

# select PressEvent and MediaStart event, to focus on user actions
home_users = home_users[(home_users.event_type == 'PressEvent') |
                   (home_users.event_type == 'MediaStartEvent')]

# home users by feature
home_users = home_users.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
home_users.sort_values('user_id_nunique', ascending=False).head()

## What users do most in docs?

In [ ]:
# select docs RootLocation only
docs_users = of[(of.application == 'objectiv-docs')]

# select PressEvent and MediaStart event, to focus on user actions
docs_users = docs_users[(docs_users.event_type == 'PressEvent') |
                   (docs_users.event_type == 'MediaStartEvent')]

# docs users by feature
docs_users = docs_users.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
docs_users.sort_values('user_id_nunique', ascending=False).head()

## Where are users coming from?

In [ ]:
# users by referrer
referrer_users = of.groupby(['referrer']).agg({'user_id':'nunique'})
referrer_users.sort_values('user_id_nunique', ascending=False).head()

## How are conversions doing?

In [ ]:
# define which events to use as conversion events
of.add_conversion_event(location_stack=of.location_stack.json[{'id': 'objectiv-on-github', 
                                                        '_type': 'LinkContext'}:].fillna(
                                       of.location_stack.json[{'id': 'github', '_type': 'LinkContext'}:]),
                        event_type='PressEvent',
                        name='github_press')

In [ ]:
# model hub: calculate conversions
conversions = of.model_hub.filter(of.model_hub.map.is_conversion_event('github_press'))\
                .model_hub.aggregate.unique_users()

conversions.sort_index(ascending=False).head(10)

In [ ]:
# use earlier model hub outputs to calculate conversion rate
conversion_rate = users.to_frame().merge(conversions.to_frame(), how='inner', on='moment', suffixes=['_total', '_converted'])
conversion_rate['conversion_rate'] = conversion_rate['unique_users_converted'] / conversion_rate['unique_users_total']

conversion_rate.sort_index(ascending=False).head(10)

In [ ]:
# from where do users convert most?
conversion_locations = of.model_hub.filter(of.model_hub.map.is_conversion_event('github_press'))\
    .groupby(['application', 'feature_nice_name', 'event_type'])\
    .agg({'user_id':'nunique'})

conversion_locations.sort_values('user_id_nunique', ascending=False).head()

In [ ]:
# what are users doing before they convert?

# select sessions with a conversion
converted_users = of.model_hub.filter(of.model_hub.map.conversions_counter(name='github_press')>=1)

# from those, select hits where number of conversions was still 0
converted_users = converted_users.mh.filter(converted_users.model_hub.map.conversions_in_time('github_press')==0)

# select PressEvent and MediaStart event, to focus on user actions
converted_users = converted_users[(converted_users.event_type == 'PressEvent') |
                   (converted_users.event_type == 'MediaStartEvent')]

converted_users.groupby(['application', 'feature_nice_name', 'event_type']).agg({'user_id':'nunique'})\
    .sort_values('user_id_nunique', ascending=False).head(10)

In [ ]:
# how much time do users spent before they convert?
converted_users.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()